<a href="https://colab.research.google.com/github/alexunderch/Master_materials/blob/main/gpaphs_and_nets/%D0%9C%D0%A2%D0%98%D0%98_%D0%B3%D1%80%D0%B0%D1%84%D1%8B_%D0%B4%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 2

In [ ]:
import networkx as nx

### Задание 1
Построив соответствующую сеть и решив для неё задачу о максимальном потоке любым из [имеющихся в networkx алгоритмов](https://networkx.org/documentation/stable/reference/algorithms/flow.html), найдите паросочетание максимального размера в заданном неориентированном двудольном графе $G$ ниже. Убедитесь, что Ваш код проходит тест ниже.

In [ ]:
k = 5
n = 20
G = nx.Graph()
G.add_nodes_from(range(2 * n))
G.add_edges_from(
    (a, b + n) 
    for a, b in nx.random_regular_graph(k, n, 2022).edges()
)
# Построенный выше тестовый граф G — двудольный
# с номерами вершин в одной доле от 0 до n-1
# и в другой доле от n до 2*n-1

In [ ]:
try:
    bottom_nodes, top_nodes = nx.bipartite.sets(G)
except Exception as e:
    print(e)

Disconnected graph: Ambiguous solution for bipartite sets.


In [ ]:
G_net = nx.DiGraph()
for e in G.edges:
    #now the graph is connected
    G_net.add_edge(*e, capacity = 1.)

bottom_nodes, top_nodes = nx.bipartite.sets(G_net)

#adding source and all edges to the bottom nodes
G_net.add_node(0)
for v in bottom_nodes:
    G_net.add_edge(0, v, capacity = 1.)

#adding target and all edges from the top nodes
t = max(top_nodes) + 1
G_net.add_node(t)
for v in top_nodes:
    G_net.add_edge(v, t, capacity = 1.)


In [ ]:
from networkx.algorithms.flow import edmonds_karp
_, flow_dict = nx.maximum_flow(G_net, 0, t, flow_func=edmonds_karp)
#filtering nodes without flow in or out
M = {k: v for k, v in flow_dict.items() if v != {} and k != 0 and k != t} 
M = [(k, subk) for k, v in M.items() for subk, subv in v.items() if subv == 1.0 and subk != t]


In [ ]:
# Это простая проверка того, что M образует паросочетание,
# и что его размер совпадает с эталонным
from functools import reduce
print(1 == len({
                    2 * len(M),
                    len(reduce(lambda e1, e2: set(e1) | set(e2), M)),
                    len(nx.bipartite.maximum_matching(G, range(n)))
                }
               )
    )

True


### Задание 2
Построив соответствующую сеть и решив для неё задачу о максимальном потоке, найдите максимально возможное количество не пересекающихся по внутренним **вершинам** путей между вершинами $s$ и $t$ в заданном неориентированном графе $G$. Убедитесь, что Ваш код проходит тест ниже.

In [ ]:
def num_disjoint_paths(G: nx.Graph, s: int, t: int) -> float:
    """
    It is required to build a flow in a given node-connected graph
    max flow value = min cut capacity = number of disjoint by vertexes paths
    """
    def build_node_connected_net(G: nx.Graph) -> nx.DiGraph:
        directed = G.is_directed()
        G_net = nx.DiGraph()
        #to remember the original node mapping
        mapping = dict()
        #duplicating every vertex V = (Vminus, Vplus)
        for i, node in enumerate(G):
            mapping[node] = i
            G_net.add_node(f"{i}minus", id = node)
            G_net.add_node(f"{i}plus", id = node)
            G_net.add_edge(f"{i}minus", f"{i}plus", capacity = 1.)

        edges = []
        #creating edges like (Vplus, Uminus)
        for (u, v) in G.edges():
            edges.append((f"{mapping[u]}plus", f"{mapping[v]}minus"))
            if not directed:
                edges.append((f"{mapping[v]}plus", f"{mapping[u]}minus"))
        G_net.add_edges_from(edges, capacity = 1.)

        # Store the mapping as graph attribute
        G_net.graph["mapping"] = mapping
        return G_net
    H = build_node_connected_net(G)

    #these nodes are not necessary to be in the net -- they are redundant
    H.remove_nodes_from([f"{s}minus", f"{t}plus"])
    
    return nx.maximum_flow_value(H, 
                                _s = f"{s}plus", _t = f"{t}minus", 
                                capacity = "capacity")


In [ ]:
# Простой тест, он должен проходиться при запуске непосредственно после предыдущей ячейки
G = nx.mycielski_graph(10)
s = 7
t = 700
print(
    num_disjoint_paths(G, s, t) 
    == 
    nx.node_connectivity(G, s, t)
)


True


### Задание 3
Построив поток в соответствующей сети, выберите из данного множества задач `tasks` такое подмножество, суммарная выгода выполнения задач которого максимальна. 

Словарь `prerequisites` содержит для каждой задачи перечень других задач, которые требуется решить перед началом её выполнения (если таковые есть). 

Словарь `profit` содержит информацию о выгоде решения каждой из задач. Выгоды — целочисленные, могут быть как положительные, так и отрицательные, и нулевые.

Также задано множество `compulsory` задач, которые нужно решить в обязательном порядке. Подумайте, и опишите в комментириях, как с минимальными усилиями интегрировать это в рассказанный на лекции алгоритм. Убедитесь, что Ваш код проходит тест ниже.

In [ ]:
# Ниже приведён алгоритм решения задачи полным перебором,
# результат работы которого можно считать эталонным
from itertools import combinations,combinations_with_replacement

def brute_force_selection(tasks, prerequisites, profit, compulsory):
    best_profit = 0
    best_selection = set()
    for ss in range(1, len(tasks)):
        for c in combinations(tasks, ss):
            if any(t not in c for t in compulsory):
                continue
            for t in c:
                if t in prerequisites:
                    for p in prerequisites[t]:
                        if p not in c:
                            break
                    else:
                        continue
                    break
            else:
                current_profit = sum(profit[t] for t in c)
                if current_profit > best_profit:
                    best_profit = current_profit
                    best_selection = set(c)
    return best_selection

In [ ]:
from functools import reduce
from networkx.algorithms.flow import edmonds_karp
def flow_based_selection(tasks: set, prerequisites: dict, profit: dict, compulsory: set) -> set:
    c = 100

    #creating an auxillary net
    G = nx.DiGraph()
    s = min(tasks) - 1
    t = max(tasks) + 1
    
    #adding nodes of proritable and non-profitable tasks
    prof, nonprof = set(), set()
    G.add_nodes_from([s, t])
    for task in tasks:
        if profit[task] > 0:
            prof.add(task)
            G.add_edge(s, task, capacity = profit[task])
        else:
            nonprof.add(task)
            G.add_edge(task, t, capacity = -profit[task])
    

    #adding interconnected prerequisite projects
    for i in prerequisites:
        sum_c = 0
        for j in prerequisites[i]:
            sum_c += abs(profit[j])
            G.add_edge(i, j, capacity = sum_c + c)

    #adding сompulsory set
    for r in compulsory:
        G.add_edge(s, r, capacity = c)
        
    #this should include all necessary projects 
    cut_value, partition = nx.flow.minimum_cut(G, s, t, capacity = "capacity")
    result = partition[0]
    result.remove(0)
    return result



In [ ]:
# Простой тест, он должен проходиться при запуске непосредственно после выполнения предыдущей ячейки
import random
def test(seeds: list):
    result = []
    for seed in seeds:
        random.seed(seed)

        tasks = set(range(1, 22))

        ranges = [
            (1, len(tasks) // 3), 
            (len(tasks) // 3, 2 * len(tasks) // 3), 
            (2 * len(tasks) // 3, len(tasks))
        ]

        prerequisites = {}
        for ri, r in enumerate(ranges):
            if ri == 0:
                continue
            for t in range(*r):
                if random.randrange(1000)/1000 < 0.4:
                    prerequisites[t] = set(random.sample(range(*ranges[ri-1]), 4))

        profit = {
            task: random.randrange(-7, 15)
            for task in tasks
        }

        compulsory = set(random.sample(tasks, 2))
        result.append(1 == len(set(
                                    sum(map(profit.get, 
                                        method(
                                            tasks, 
                                            prerequisites, 
                                            profit, 
                                            compulsory
                                        )
                                    ))
                                    for method in [
                                        brute_force_selection, 
                                        flow_based_selection
                                    ]
                                )))
    print(all(result))

test([1234, 42, 2022, 6969])

True
